<br><br>
**INTRO**

In [1]:
import sys.process._

import sys.process._

In [2]:
"ls -l" !

total 6712
-rwxrwxrwx 1 jovyan 1000    2083 Sep 12 17:33 about
-rwxrwxrwx 1 jovyan 1000 6794338 Sep 16 13:52 breeze_linreg.ipynb
drwxrwxrwx 4 jovyan 1000    4096 Sep 12 17:26 data
-rwxrwxrwx 1 jovyan 1000   66102 Sep 16 13:52 spark_sna_eda.ipynb


res1: Int = 0

In [3]:
def wd = {val dir = "pwd" !!; dir.stripLineEnd}

defined function wd

In [4]:
// 1. Скачайте датасет

val parquet_train = f"${wd}/data/SNAHackathon2019/collabTrain"
val parquet_test = f"${wd}/data/SNAHackathon2019/collabTest"

parquet_train: String = "/home/jovyan/work/spark_homework/data/SNAHackathon2019/collabTrain"
parquet_test: String = "/home/jovyan/work/spark_homework/data/SNAHackathon2019/collabTest"

In [5]:
f"du -s -h $parquet_train " !

2.2G	/home/jovyan/work/spark_homework/data/SNAHackathon2019/collabTrain


res4: Int = 0

In [6]:
f"du -s -h $parquet_test " !

126M	/home/jovyan/work/spark_homework/data/SNAHackathon2019/collabTest


res5: Int = 0

<br><br>
**IMPORT LIBRARY AND CREATE SPARK SESSION**

In [7]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`org.apache.spark::spark-mllib:2.4.5`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import $ivy.$                                  

import $ivy.$                                    


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

In [8]:
val spark = AmmoniteSparkSession
    .builder()
    .master("local[*]")
    .getOrCreate()

import spark.implicits._

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@609e59cd
import spark.implicits._

In [11]:
// val spark = NotebookSparkSession
//     .builder()
//     .master("local[*]")
//     .getOrCreate()

// import spark.implicits._

In [11]:
// spark.close()
// spark.stop()

In [11]:
// 2. Ознакомьтесь с его описанием

**LOAD DATA**
<br><br>
<font size="2.0">
instanceId_userId — идентификатор пользователя (анонимизированный);  
instanceId_objectType — тип объекта;  
instanceId_objectId — идентификатор объекта (анонимизированный);  
feedback — массив с типами реакций пользователя (наличие в массиве токена Liked говорит о том, что объект получил «класс» от пользователя);  
audit_clientType — тип платформы, с которой зашёл пользователь;  
audit_timestamp — время, когда строилась лента;  
metadata_ownerId — автор показанного объекта (анонимизированный);  
metadata_createdAt — дата создания показанного объекта.  
</font>
<br><font size="2.0">
audit_* — расширенная информация о контексте построения ленты;  
metadata_* — расширенная информация о самом объекте;  
userOwnerCounters_* — информация о предыдущих взаимодействиях пользователя и автора контента;  
ownerUserCounters_* — информация о предыдущих взаимодействиях автора контента и пользователя;  
membership_* — информация о членстве пользователя в группе, где опубликован контент;  
user_* — подробная информация о пользователе;  
auditweights_* — большое количество runtime-признаков, извлечённых текущей системой.  
</font>

In [9]:
// load from parquet
val df = spark.read.parquet(parquet_train).cache()

// get dimention of dataframe
val shape = (df.count().toInt, df.columns.length)

// group attribute by prefix name
def colName = df.columns.map(col(_)).groupBy(_.toString.split("_")(0))

// group attribute by spark type
def colType = (
    df
    .schema
    .fields.map(f => (col(f.name), f.dataType.toString))
    .groupBy(_._2)
    .mapValues(x => x.map(y => y._1))
)

df: Dataset[Row] = [instanceId_userId: int, instanceId_objectType: string ... 167 more fields]
shape: (Int, Int) = (18286575, 169)
defined function colName
defined function colType

In [10]:
df.printSchema

root
 |-- instanceId_userId: integer (nullable = true)
 |-- instanceId_objectType: string (nullable = true)
 |-- instanceId_objectId: integer (nullable = true)
 |-- audit_pos: long (nullable = true)
 |-- audit_clientType: string (nullable = true)
 |-- audit_timestamp: long (nullable = true)
 |-- audit_timePassed: long (nullable = true)
 |-- audit_experiment: string (nullable = true)
 |-- audit_resourceType: long (nullable = true)
 |-- metadata_ownerId: integer (nullable = true)
 |-- metadata_ownerType: string (nullable = true)
 |-- metadata_createdAt: long (nullable = true)
 |-- metadata_authorId: integer (nullable = true)
 |-- metadata_applicationId: long (nullable = true)
 |-- metadata_numCompanions: integer (nullable = true)
 |-- metadata_numPhotos: integer (nullable = true)
 |-- metadata_numPolls: integer (nullable = true)
 |-- metadata_numSymbols: integer (nullable = true)
 |-- metadata_numTokens: integer (nullable = true)
 |-- metadata_numVideos: integer (nullable = true)
 |-- me

In [11]:
colName.foreach {case (key, value) => println (f"$key: ${value.length}")}

instanceId: 3
membership: 4
relationsMask: 1
userOwnerCounters: 27
feedback: 1
date: 1
metadata: 14
objectId: 1
audit: 6
owner: 13
ownerUserCounters: 27
auditweights: 58
user: 13


In [12]:
df.select(colName("instanceId"): _*).show(3)

+-----------------+---------------------+-------------------+
|instanceId_userId|instanceId_objectType|instanceId_objectId|
+-----------------+---------------------+-------------------+
|              108|                 Post|           18452434|
|              231|                 Post|           31980032|
|              423|                 Post|           33834009|
+-----------------+---------------------+-------------------+
only showing top 3 rows



In [13]:
colType.foreach {case (key, value) => println (f"$key: ${value.length}")}

StringType: 6
LongType: 17
IntegerType: 32
ArrayType(StringType,true): 2
DoubleType: 111
DateType: 1


In [14]:
df.select(colType("StringType"): _*).show(3)

+---------------------+----------------+----------------+-------------------+-----------------+-----------------+
|instanceId_objectType|audit_clientType|audit_experiment| metadata_ownerType|metadata_platform|membership_status|
+---------------------+----------------+----------------+-------------------+-----------------+-----------------+
|                 Post|             MOB|    XPRM-5386_G1|GROUP_OPEN_OFFICIAL|            OTHER|                A|
|                 Post|             MOB|    XPRM-5386_G2|GROUP_OPEN_OFFICIAL|            OTHER|                A|
|                 Post|             API|    XPRM-5386_G9|         GROUP_OPEN|          ANDROID|             null|
+---------------------+----------------+----------------+-------------------+-----------------+-----------------+
only showing top 3 rows



In [15]:
df.select(colType("ArrayType(StringType,true)"): _*).show(3)

+--------------------+----------------+
|    metadata_options|        feedback|
+--------------------+----------------+
|[HAS_TEXT, HAS_PH...|         [Liked]|
|[HAS_TEXT, HAS_PH...|[Clicked, Liked]|
|[IS_PART_OF_TOPIC...|         [Liked]|
+--------------------+----------------+
only showing top 3 rows



In [15]:
// summary report - data audit : TODO

In [18]:
// 3. Постройте топ популярных групп на портале

In [19]:
// Посмотрим, что лежит в feedback

(
    df
    .select("feedback")
    .withColumn("feedback", explode($"feedback"))
    .groupBy("feedback")
    .count()
    .orderBy(desc("count"))
    .withColumn("fraction", round($"count"/ shape._1, 2))
    .show()
)


+---------+--------+--------+
| feedback|   count|fraction|
+---------+--------+--------+
|  Ignored|12421394|    0.68|
|    Liked| 3220429|    0.18|
|  Clicked| 2258630|    0.12|
| Disliked| 1665403|    0.09|
| ReShared|  321631|    0.02|
|   Viewed|  170031|    0.01|
|Commented|   36244|     0.0|
|  Unliked|   33428|     0.0|
|Complaint|     263|     0.0|
+---------+--------+--------+



In [ ]:
// spark.catalog.clearCache()
// spark.close()
// spark.stop()

In [20]:
// Рейтинг

val colSeq = Seq("date", "hour", "metadata_authorId", "instanceId_objectId", "instanceId_userId", "feedback")

val out = df

.withColumn("hour", hour(($"audit_timestamp"/1000).cast(types.TimestampType)) )
.select(colSeq.map(col(_)): _*)
.withColumnRenamed("metadata_authorId", "authorId")
.withColumnRenamed("instanceId_objectId", "objectId")
.withColumnRenamed("instanceId_userId", "userId")
.withColumn("Liked", ( array_contains($"feedback", "Liked")).cast(types.IntegerType) )
.withColumn("Disliked", ( array_contains($"feedback", "Disliked")).cast(types.IntegerType) )
.withColumn("Interacted", ( !array_contains($"feedback", "Ignored")).cast(types.IntegerType) )

println("SAMPLE")
out.show(10)

val out1 = out

.groupBy("authorId")
.agg(
    count("*").alias("baseAll"), 
    countDistinct("objectId").alias("objShare"),
    countDistinct("userId").alias("usrShare"),
    sum("Liked").alias("lik"),
    sum("Disliked").alias("dis"),
    sum("Interacted").alias("act")
)

// baseAll       - сколько всего было фактов комуникации от данного Автора
// objShare      - сколько уникальных объектов было показано
// usrShare      - сколько уникальных юзеров это увидело
// lik|dis|act   - результат коммуникации относительно baseAll

println("GROUP")
out1.show(10)

// расчитаем удельные промежуточные метрики

val cntAll = out.count()
val uniqObj = out.select($"objectId").groupBy("objectId").count().first().getInt(0)
val uniqUsr = out.select($"userId").groupBy("userId").count().first().getInt(0)
// val minBase = 100

val out2 = out1
// .filter($"baseAll" >= minBase)

.withColumn( "shareAll", round($"baseAll"*100 / cntAll, 4).cast(types.DoubleType) )
.withColumn( "shareObj", round($"objShare"*100 / uniqObj, 4).cast(types.DoubleType) )
.withColumn( "shareUsr", round($"usrShare"*100 / uniqUsr, 4).cast(types.DoubleType) )
.withColumn( "likesMore", round(($"lik" - $"dis")*100 / $"baseAll", 2) )
.withColumn( "interMore", round($"act"*100 / $"baseAll", 2) )

.select('authorId, 'shareAll, 'shareObj, 'shareUsr, 'likesMore, 'interMore)

println("PROXY")
out2.show(10)

def specWindow(colName: String) = org.apache.spark.sql.expressions.Window.orderBy(colName)

val out3 = out2
.withColumn("R1", rank().over(specWindow("shareAll")))
.withColumn("R2", rank().over(specWindow("shareObj")))
.withColumn("R3", rank().over(specWindow("shareUsr")))
.withColumn("R4", rank().over(specWindow("likesMore")))
.withColumn("R5", rank().over(specWindow("interMore")))

// formula from my head = 5 10 10 50 25
val out4 = out3
.withColumn("avgR", round(
    (
        out3("R1")*5+out3("R2")*10+out3("R3")*10+out3("R4")*50+out3("R5")*25
    )/500, 2)
            .cast(types.DoubleType))

.orderBy(desc("avgR"))

println("RANKING")
out4.show(10)



SAMPLE
+----------+----+--------+--------+------+----------------+-----+--------+----------+
|      date|hour|authorId|objectId|userId|        feedback|Liked|Disliked|Interacted|
+----------+----+--------+--------+------+----------------+-----+--------+----------+
|2018-03-04|  20|   73356|18452434|   108|         [Liked]|    1|       0|         1|
|2018-03-04|  21|  828613|31980032|   231|[Clicked, Liked]|    1|       0|         1|
|2018-03-04|  16| 1076820|33834009|   423|         [Liked]|    1|       0|         1|
|2018-03-04|   7|  608137|25653019|   624|       [Ignored]|    0|       0|         0|
|2018-03-04|  16|  565604|24024934|   768|       [Ignored]|    0|       0|         0|
|2018-03-04|  10|  981198|25075524|  1206|       [Ignored]|    0|       0|         0|
|2018-03-04|   7|  579806|18132300|  1752|[Clicked, Liked]|    1|       0|         1|
|2018-03-04|   7|  200111| 2402728|  1752|         [Liked]|    1|       0|         1|
|2018-03-04|  10|  980484|17153719|  1752|     

colSeq: Seq[String] = List(
  "date",
  "hour",
  "metadata_authorId",
  "instanceId_objectId",
  "instanceId_userId",
  "feedback"
)
out: DataFrame = [date: date, hour: int ... 7 more fields]
out1: DataFrame = [authorId: int, baseAll: bigint ... 5 more fields]
cntAll: Long = 18286575L
uniqObj: Int = 12871510
uniqUsr: Int = 70863
out2: DataFrame = [authorId: int, shareAll: double ... 4 more fields]
defined function specWindow
out3: DataFrame = [authorId: int, shareAll: double ... 9 more fields]
out4: Dataset[Row] = [authorId: int, shareAll: double ... 10 more fields]

<br><br>
**IMPORT LIBRARY FOR VISUALIZE**

In [22]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.6`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


In [23]:
// 4. Построите гистограммы популярности/активности групп на портале по времени суток

val hist1 = (
    out
    .select('hour, 'Interacted)
    .groupBy('hour)
    .agg(sum('Interacted).alias("Interacted"))
    .orderBy('hour)
).cache()

hist1.show(24)

val hist = hist1.collect().map(_.toSeq.map(_.toString).map(_.toInt).toVector).toVector.transpose

hist.foreach(println)

+----+----------+
|hour|Interacted|
+----+----------+
|   0|     55733|
|   1|     76503|
|   2|    107668|
|   3|    157802|
|   4|    208885|
|   5|    251970|
|   6|    267698|
|   7|    272816|
|   8|    275905|
|   9|    279034|
|  10|    288987|
|  11|    299312|
|  12|    315781|
|  13|    336393|
|  14|    365712|
|  15|    395531|
|  16|    414272|
|  17|    410942|
|  18|    378458|
|  19|    290172|
|  20|    190048|
|  21|    109330|
|  22|     65414|
|  23|     50815|
+----+----------+

Vector(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23)
Vector(55733, 76503, 107668, 157802, 208885, 251970, 267698, 272816, 275905, 279034, 288987, 299312, 315781, 336393, 365712, 395531, 414272, 410942, 378458, 290172, 190048, 109330, 65414, 50815)


hist1: Dataset[Row] = [hour: int, Interacted: bigint]
hist: Vector[Vector[Int]] = Vector(
  Vector(
...

### https://alexarchambault.github.io/plotly-scala/

In [24]:
val h0 = hist(0)
val h1 = hist(1).map(x =>(x/1000).toInt)

val trace1 = Bar( h0, h1, marker = Marker(opacity = 0.6, line = Line(width = 1.5)))

val data = Seq(trace1)

val annotations = h0.zip(h1).map {
  case (x, y) =>
    Annotation(
      x = x,
      y = y,
      text = y.toString,
      xanchor = Anchor.Center,
      yanchor = Anchor.Bottom,
      showarrow = false
    )
}

val layout = Layout(
  title = "(1k) Interaction (Not Ignored) x Time",
  annotations = annotations
)

val plt = plot(data, layout)


h0: Vector[Int] = Vector(
  0,
...
h1: Vector[Int] = Vector(
  55,
...
trace1: Bar = Bar(
  Doubles(
...
data: Seq[Bar] = List(
  Bar(
...
annotations: Vector[Annotation] = Vector(
  Annotation(
...
layout: Layout = Layout(
  Some("(1k) Interaction (Not Ignored) x Time"),
...
plt: String = "plot-fd8c7a72-fdfd-40d4-af49-ab36f55a542a"

![title](img/hist1.png)

In [26]:
// 5. Посчитайте корреляцию признаков с целевой переменной

// TODO: make distibuted 
// TODO: реализовать через Statistics.corr with Spearman`s Ro
// TODO: compaired group target=0 vs target=1 with criteria

colType.foreach {case (key, value) => println (f"$key: ${value.length}")}

def corrWith_target(Type: String, df: org.apache.spark.sql.DataFrame) = {

    val df1 = df.withColumn("target", array_contains($"feedback", "Liked").cast(types.IntegerType))
    .select(Array($"target") ++ colType(Type): _*)
    
    val corr = df1.columns.map
        {
            x => x -> df1.filter(f"$x is not null").stat.corr("target", x.toString)
        }

    corr.filterNot(_._2.isNaN).map( x => ( x._1, rnd(x._2, 5) ) )
    
}

def rnd(v: Double, p: Int) = BigDecimal(v).setScale(p, BigDecimal.RoundingMode.HALF_UP).toDouble


StringType: 6
LongType: 17
IntegerType: 32
ArrayType(StringType,true): 2
DoubleType: 111
DateType: 1


defined function corrWith_target
defined function rnd

In [27]:
val corr_1 = corrWith_target("LongType", df)

corr_1: Array[(String, Double)] = Array(
  ("target", 1.0),
...

In [28]:
corr_1.foreach(println)

(target,1.0)
(audit_pos,0.00576)
(audit_timestamp,-0.00196)
(audit_timePassed,0.01353)
(audit_resourceType,0.04557)
(metadata_createdAt,7.3E-4)
(membership_statusUpdateDate,0.04154)
(membership_joinDate,0.04272)
(membership_joinRequestDate,-0.00739)
(user_create_date,0.03765)
(user_ID_country,8.0E-5)
(user_change_datime,0.03554)
(auditweights_x_ActorsRelations,-0.011)


In [29]:
val corr_2 = corrWith_target("IntegerType", df)

corr_2: Array[(String, Double)] = Array(
  ("target", 1.0),
...

In [30]:
corr_2.foreach(println)

(target,1.0)
(instanceId_userId,0.03197)
(instanceId_objectId,0.03876)
(metadata_ownerId,0.01168)
(metadata_authorId,0.00723)
(metadata_numCompanions,-0.00485)
(metadata_numPhotos,0.01119)
(metadata_numPolls,0.08115)
(metadata_numSymbols,-0.03328)
(metadata_numTokens,-0.03329)
(metadata_numVideos,-0.01294)
(metadata_totalVideoLength,-0.01438)
(user_birth_date,0.02692)
(user_gender,0.00887)
(user_status,-0.01412)
(user_ID_Location,0.00155)
(user_is_active,-0.00208)
(user_is_deleted,3.1E-4)
(user_is_abused,0.00236)
(user_is_activated,-1.9E-4)
(user_is_semiactivated,-9.3E-4)
(user_region,-0.01235)
(objectId,0.03876)


In [31]:
val corr_3 = corrWith_target("DoubleType", df)

corr_3: Array[(String, Double)] = Array(
  ("target", 1.0),
...

In [32]:
corr_3.foreach(println)

(target,1.0)
(userOwnerCounters_USER_FEED_REMOVE,-0.07256)
(userOwnerCounters_UNKNOWN,-0.00269)
(userOwnerCounters_CREATE_TOPIC,7.1E-4)
(userOwnerCounters_CREATE_IMAGE,-0.00105)
(userOwnerCounters_CREATE_MOVIE,8.8E-4)
(userOwnerCounters_CREATE_COMMENT,-9.9E-4)
(userOwnerCounters_CREATE_LIKE,0.09825)
(userOwnerCounters_TEXT,-0.0119)
(userOwnerCounters_IMAGE,-0.01089)
(userOwnerCounters_VIDEO,-0.00842)
(auditweights_ageMs,0.02956)
(auditweights_ctr_gender,0.00303)
(auditweights_ctr_high,0.14549)
(auditweights_ctr_negative,-7.0E-4)
(auditweights_dailyRecency,-0.0316)
(auditweights_feedOwner_RECOMMENDED_GROUP,0.00669)
(auditweights_feedStats,0.04412)
(auditweights_friendCommentFeeds,-1.9E-4)
(auditweights_friendCommenters,0.00449)
(auditweights_friendLikes,0.0077)
(auditweights_friendLikes_actors,0.01463)
(auditweights_likersFeedStats_hyper,0.04153)
(auditweights_likersSvd_prelaunch_hyper,0.12758)
(auditweights_matrix,0.01874)
(auditweights_numDislikes,0.00251)
(auditweights_numLikes,0.013

In [ ]:
// корреляции - говно, нужно предобрабатывать данные, да и корреляция с бинарным признаком - плохая идея

In [16]:
spark.catalog.clearCache()
spark.close()
spark.stop()

In [ ]:
// 6. Найдите другие интересные инсайты в данных :)
// TODO

// 1 балл - за каждый дополнительный интересный вывод, построенный на данных датасета

// а баллы которые начисляют за задания на что в конце обменять можно?
// интересно что за баллы такие ...


In [ ]:
// (
//     df
//     .select($"date", $"audit_timestamp")
//     .withColumn("audit_hhmm", hour( ($"audit_timestamp"/1000).cast(types.TimestampType))*100 + minute( ($"audit_timestamp"/1000).cast(types.TimestampType) ) )
//     .withColumn("audit_timestamp", ($"audit_timestamp"/1000).cast(types.TimestampType))
//     .show(3, false)
// )

In [ ]:
// import org.apache.spark.mllib.linalg._
// import org.apache.spark.mllib.stat.Statistics
// import org.apache.spark.rdd.RDD

// val sc = spark.sparkContext

// val seriesX: RDD[Double] = sc.parallelize(Array(1, 2, 3, 3, 5))  // a series
// // must have the same number of partitions and cardinality as seriesX
// val seriesY: RDD[Double] = sc.parallelize(Array(11, 22, 33, 33, 555))

// // compute the correlation using Pearson's method. Enter "spearman" for Spearman's method. If a
// // method is not specified, Pearson's method will be used by default.
// val correlation: Double = Statistics.corr(seriesX, seriesY, "pearson")
// println(s"Correlation is: $correlation")

// val data: RDD[Vector] = sc.parallelize(
//   Seq(
//     Vectors.dense(1.0, 10.0, 100.0),
//     Vectors.dense(2.0, 20.0, 200.0),
//     Vectors.dense(5.0, 33.0, 366.0))
// )  // note that each Vector is a row and not a column

// // calculate the correlation matrix using Pearson's method. Use "spearman" for Spearman's method
// // If a method is not specified, Pearson's method will be used by default.
// val correlMatrix: Matrix = Statistics.corr(data, "pearson")
// println(correlMatrix.toString)
